# EFI Graph Execution Example

In [ ]:
from cascade.cascade import Cascade

from ppcascade.entry.parser import get_parser

parser = get_parser("extreme")
graph = Cascade.graph("extreme", parser.parse_args(["-c", "efi.yaml", "--forecast", "mars:ens", "--climatology", "mars:clim"]))

In [ ]:
import functools

from cascade.graph import pyvis

def node_info_ext(sinks, node):
    info = pyvis.node_info(node)
    info["color"] = "#648FFF"
    if not node.inputs:
        info["shape"] = "diamond"
        info["color"] = "#DC267F"
    elif node in sinks:
        info["shape"] = "triangle"
        info["color"] = "#FFB000"
    if node.payload is not None:
        t = []
        if "title" in info:
            t.append(info["title"])
        func, *args = node.payload
        t.append(f"Function: {func}")
        if args:
            t.append("Arguments:")
            t.extend(f"- {arg!r}" for arg in args)
        info["title"] = "\n".join(t)
    return info

pyvis_graph = pyvis.to_pyvis(
        graph,
        notebook=True,
        cdn_resources="remote",
        height="1500px",
        node_attrs=functools.partial(node_info_ext, graph.sinks),
        hierarchical_layout=False,
    )
pyvis_graph.show(f"efi_graph.html")

In [ ]:
from cascade.executors.dask import DaskLocalExecutor 

DaskLocalExecutor.execute(
    graph,
    n_workers=2,
    threads_per_worker=1,
    memory_limit="10GB",
    report=f"efi_dask_report.html",
)
